## Mailing List Data - Part I
In this assignment you will download some of the mailing list data from http://mbox.dr-chuck.net/ and run the data cleaning / modeling process and take some screen shots. All screenshots should use the same data set.

Don't take off points for little mistakes. If they seem to have done the assignment give them full credit. Feel free to make suggestions if there are small mistakes. Please keep your comments positive and useful. The course staff will assign the last 30% of the grade and it make take a few days to get graded.
<img src="./Captura.PNG" ></img>



#### Primer código "gmane.py"

In [1]:
import sqlite3
import time
import ssl
import urllib.request, urllib.parse, urllib.error
from urllib.parse import urljoin
from urllib.parse import urlparse
import re
from datetime import datetime, timedelta

# Not all systems have this so conditionally define parser
try:
    import dateutil.parser as parser
except:
    pass

def parsemaildate(md) :
    # See if we have dateutil
    try:
        pdate = parser.parse(tdate)
        test_at = pdate.isoformat()
        return test_at
    except:
        pass

    # Non-dateutil version - we try our best

    pieces = md.split()
    notz = " ".join(pieces[:4]).strip()

    # Try a bunch of format variations - strptime() is *lame*
    dnotz = None
    for form in [ '%d %b %Y %H:%M:%S', '%d %b %Y %H:%M:%S',
        '%d %b %Y %H:%M', '%d %b %Y %H:%M', '%d %b %y %H:%M:%S',
        '%d %b %y %H:%M:%S', '%d %b %y %H:%M', '%d %b %y %H:%M' ] :
        try:
            dnotz = datetime.strptime(notz, form)
            break
        except:
            continue

    if dnotz is None :
        # print 'Bad Date:',md
        return None

    iso = dnotz.isoformat()

    tz = "+0000"
    try:
        tz = pieces[4]
        ival = int(tz) # Only want numeric timezone values
        if tz == '-0000' : tz = '+0000'
        tzh = tz[:3]
        tzm = tz[3:]
        tz = tzh+":"+tzm
    except:
        pass

    return iso+tz

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('content.sqlite')
cur = conn.cursor()

baseurl = "http://mbox.dr-chuck.net/sakai.devel/"

cur.execute('''CREATE TABLE IF NOT EXISTS Messages
    (id INTEGER UNIQUE, email TEXT, sent_at TEXT,
     subject TEXT, headers TEXT, body TEXT)''')

# Pick up where we left off
start = None
cur.execute('SELECT max(id) FROM Messages' )
try:
    row = cur.fetchone()
    if row is None :
        start = 0
    else:
        start = row[0]
except:
    start = 0

if start is None : start = 0

many = 0
count = 0
fail = 0
while True:
    if ( many < 1 ) :
        conn.commit()
        sval = input('How many messages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)

    start = start + 1
    cur.execute('SELECT id FROM Messages WHERE id=?', (start,) )
    try:
        row = cur.fetchone()
        if row is not None : continue
    except:
        row = None

    many = many - 1
    url = baseurl + str(start) + '/' + str(start + 1)

    text = "None"
    try:
        # Open with a timeout of 30 seconds
        document = urllib.request.urlopen(url, None, 30, context=ctx)
        text = document.read().decode()
        if document.getcode() != 200 :
            print("Error code=",document.getcode(), url)
            break
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except Exception as e:
        print("Unable to retrieve or parse page",url)
        print("Error",e)
        fail = fail + 1
        if fail > 5 : break
        continue

    print(url,len(text))
    count = count + 1

    if not text.startswith("From "):
        print(text)
        print("Did not find From ")
        fail = fail + 1
        if fail > 5 : break
        continue

    pos = text.find("\n\n")
    if pos > 0 :
        hdr = text[:pos]
        body = text[pos+2:]
    else:
        print(text)
        print("Could not find break between headers and body")
        fail = fail + 1
        if fail > 5 : break
        continue

    email = None
    x = re.findall('\nFrom: .* <(\S+@\S+)>\n', hdr)
    if len(x) == 1 :
        email = x[0];
        email = email.strip().lower()
        email = email.replace("<","")
    else:
        x = re.findall('\nFrom: (\S+@\S+)\n', hdr)
        if len(x) == 1 :
            email = x[0];
            email = email.strip().lower()
            email = email.replace("<","")

    date = None
    y = re.findall('\Date: .*, (.*)\n', hdr)
    if len(y) == 1 :
        tdate = y[0]
        tdate = tdate[:26]
        try:
            sent_at = parsemaildate(tdate)
        except:
            print(text)
            print("Parse fail",tdate)
            fail = fail + 1
            if fail > 5 : break
            continue

    subject = None
    z = re.findall('\Subject: (.*)\n', hdr)
    if len(z) == 1 : subject = z[0].strip().lower();

    # Reset the fail counter
    fail = 0
    print("   ",email,sent_at,subject)
    cur.execute('''INSERT OR IGNORE INTO Messages (id, email, sent_at, subject, headers, body)
        VALUES ( ?, ?, ?, ?, ?, ? )''', ( start, email, sent_at, subject, hdr, body))
    if count % 50 == 0 : conn.commit()
    if count % 100 == 0 : time.sleep(1)

conn.commit()
cur.close()


How many messages:500
http://mbox.dr-chuck.net/sakai.devel/1/2 2662
    ggolden@umich.edu 2005-12-08T23:34:30-06:00 call for participation: developers documentation
http://mbox.dr-chuck.net/sakai.devel/2/3 2434
    csev@umich.edu 2005-12-09T00:58:01-05:00 report from the austin conference:  sakai developers break into song
http://mbox.dr-chuck.net/sakai.devel/3/4 3055
    kevin.carpenter@rsmart.com 2005-12-09T09:01:49-07:00 cas and sakai 1.5
http://mbox.dr-chuck.net/sakai.devel/4/5 11721
    michael.feldstein@suny.edu 2005-12-09T09:43:12-05:00 re: lms/vle rants/comments
http://mbox.dr-chuck.net/sakai.devel/5/6 9443
    john@caret.cam.ac.uk 2005-12-09T13:32:29+00:00 re: lms/vle rants/comments
http://mbox.dr-chuck.net/sakai.devel/6/7 3586
    s-githens@northwestern.edu 2005-12-09T13:32:31-06:00 re: sakaiportallogin and presense
http://mbox.dr-chuck.net/sakai.devel/7/8 10600
    john@caret.cam.ac.uk 2005-12-09T13:42:24+00:00 re: lms/vle rants/comments
http://mbox.dr-chuck.net/sakai.devel/

http://mbox.dr-chuck.net/sakai.devel/63/64 3480
    iros@student.umass.edu 2005-12-13T21:19:04-05:00 presence tool and virtual paths...
http://mbox.dr-chuck.net/sakai.devel/64/65 21272
    masayang.mobile@gmail.com 2005-12-13T22:47:23-08:00 re: utf-8 problem on sakai 2.1 with mysql
http://mbox.dr-chuck.net/sakai.devel/65/66 3399
    lfernandez@weber.edu 2005-12-13T23:22:34-07:00 re: password forgotten feature
http://mbox.dr-chuck.net/sakai.devel/66/67 4536
    lfernandez@weber.edu 2005-12-13T23:43:16-07:00 re: password forgotten feature
http://mbox.dr-chuck.net/sakai.devel/67/68 5449
    cheryl.wogahn@yale.edu 2005-12-14T08:46:29-05:00 re: password forgotten feature
http://mbox.dr-chuck.net/sakai.devel/68/69 7980
    lfernandez@weber.edu 2005-12-14T08:49:28-07:00 re: password forgotten feature
http://mbox.dr-chuck.net/sakai.devel/69/70 3684
    clayf@bu.edu 2005-12-14T08:49:31-05:00 re: sakai 2.1 webdav
http://mbox.dr-chuck.net/sakai.devel/70/71 3905
    ggolden@umich.edu 2005-12-14T09

http://mbox.dr-chuck.net/sakai.devel/125/126 10124
    nuno@ufp.pt 2005-12-15T18:39:37+00:00 re: sectionmanager api's (nosuchbeandefinitionexception)
http://mbox.dr-chuck.net/sakai.devel/126/127 2621
    news_tmp1@edegger.com 2005-12-15T21:30:28-08:00 sakai:button_bar_item onclick attribute
http://mbox.dr-chuck.net/sakai.devel/127/128 4728
    ggolden@umich.edu 2005-12-15T22:49:00-05:00 sakai 2.1 development news
http://mbox.dr-chuck.net/sakai.devel/128/129 4053
    v.n.maijer@uva.nl 2005-12-15T23:15:36+01:00 re: sakai on oracle xe
http://mbox.dr-chuck.net/sakai.devel/129/130 3637
    jimeng@umich.edu 2005-12-16T04:52:00-05:00 maven bus error?
http://mbox.dr-chuck.net/sakai.devel/130/131 3053
    jleasia@umich.edu 2005-12-16T07:46:12-05:00 re: email address as username
http://mbox.dr-chuck.net/sakai.devel/131/132 3562
    ggolden@umich.edu 2005-12-16T08:19:26-05:00 re: email address as username
http://mbox.dr-chuck.net/sakai.devel/132/133 6089
    bkirschn@umich.edu 2005-12-16T10:48:29

http://mbox.dr-chuck.net/sakai.devel/187/188 3502
    jholtzman@berkeley.edu 2005-12-20T08:55:38-08:00 re: oracle problem in sakai 2.1
http://mbox.dr-chuck.net/sakai.devel/188/189 3744
    marquard@ched.uct.ac.za 2005-12-20T09:13:10+02:00 re: sakai desktop portal (in visual basic) version 03
http://mbox.dr-chuck.net/sakai.devel/189/190 3350
    slt@columbia.edu 2005-12-20T09:23:10-05:00 re: fw: how to use sakaiscript????
http://mbox.dr-chuck.net/sakai.devel/190/191 5474
    daphne@media.berkeley.edu 2005-12-20T09:29:57-08:00 re: sakai desktop portal (in visual basic) version 03
http://mbox.dr-chuck.net/sakai.devel/191/192 2823
    p.beckett@uea.ac.uk 2005-12-20T09:37:11+00:00 re: how to use sakaiscript????
http://mbox.dr-chuck.net/sakai.devel/192/193 2084
    dgcotton@ucdavis.edu 2005-12-20T10:26:33-08:00 update db 1.5.1 to 2.1
http://mbox.dr-chuck.net/sakai.devel/193/194 5386
    shaneosullivan1@gmail.com 2005-12-20T10:38:38+00:00 web services delegates and documentation
http://mbox.d

http://mbox.dr-chuck.net/sakai.devel/247/248 4004
    cheryl.wogahn@yale.edu 2005-12-22T08:46:55-05:00 re: assigning different 'my workspace' resources quota
http://mbox.dr-chuck.net/sakai.devel/248/249 2993
    tpamsler@ucdavis.edu 2005-12-22T08:54:36-08:00 sakai: uc davis hosts the next programmers boot camp (feb. 26 to feb. 28, 2006)
http://mbox.dr-chuck.net/sakai.devel/249/250 2566
    tpamsler@ucdavis.edu 2005-12-22T09:17:09-08:00 sakai  tool: message center (message forums) status
http://mbox.dr-chuck.net/sakai.devel/250/251 3646
    jholtzman@berkeley.edu 2005-12-22T09:45:21-08:00 re: sakai  tool: message center (message forums) status
http://mbox.dr-chuck.net/sakai.devel/251/252 2825
    tpamsler@ucdavis.edu 2005-12-22T10:03:20-08:00 sakai: which code base to use in production: maintenance branch or release tags
http://mbox.dr-chuck.net/sakai.devel/252/253 5138
    zqian@umich.edu 2005-12-22T10:04:12-05:00 re: assigning different 'my workspace' resources quota
http://mbox.dr-ch

http://mbox.dr-chuck.net/sakai.devel/304/305 3350
    khancock@iupui.edu 2006-01-03T08:44:18-05:00 message center permissions
http://mbox.dr-chuck.net/sakai.devel/305/306 4808
    sinouvivian@foothill.edu 2006-01-03T08:54:55-08:00 re: sakai 2.1 melete cannot upload files
http://mbox.dr-chuck.net/sakai.devel/306/307 4229
    tpamsler@ucdavis.edu 2006-01-03T09:24:14-08:00 re: sakai 2.1 melete cannot upload files
http://mbox.dr-chuck.net/sakai.devel/307/308 2549
    tpamsler@ucdavis.edu 2006-01-03T09:36:57-08:00 sakai: running multiple tomcat/jvms on a single server
http://mbox.dr-chuck.net/sakai.devel/308/309 2861
    sinouvivian@foothill.edu 2006-01-03T10:38:23-08:00 samigo in 2.1 has no default template (for instructor)
http://mbox.dr-chuck.net/sakai.devel/309/310 3144
    aaronz@vt.edu 2006-01-03T11:08:45-05:00 sakai 2.1 melete cannot upload files
http://mbox.dr-chuck.net/sakai.devel/310/311 3232
    vrajgopalan@ucmerced.edu 2006-01-03T11:14:46-08:00 performance issues in sakai 2.1
ht

http://mbox.dr-chuck.net/sakai.devel/362/363 2299
    csev@umich.edu 2006-01-06T01:07:16-05:00 is svn having problems?
http://mbox.dr-chuck.net/sakai.devel/363/364 3311
    ajpoland@iupui.edu 2006-01-06T01:31:47-05:00 re: is svn having problems?
http://mbox.dr-chuck.net/sakai.devel/364/365 5475
    lance@indiana.edu 2006-01-06T09:33:42-05:00 re: staleobjectstateexception updating profile
http://mbox.dr-chuck.net/sakai.devel/365/366 3017
    nuno@ufp.pt 2006-01-06T09:51:09+00:00 staleobjectstateexception updating profile
http://mbox.dr-chuck.net/sakai.devel/366/367 5297
    nuno@ufp.pt 2006-01-06T10:44:43+00:00 re: melete permissions problem
http://mbox.dr-chuck.net/sakai.devel/367/368 4116
    lance@indiana.edu 2006-01-06T11:10:23-05:00 heads-up: production branch changes
http://mbox.dr-chuck.net/sakai.devel/368/369 8153
    jleasia@umich.edu 2006-01-06T11:23:21-05:00 re: creating new user account types and changing tools / qouta of
http://mbox.dr-chuck.net/sakai.devel/369/370 8914
   

http://mbox.dr-chuck.net/sakai.devel/419/420 3396
    csev@umich.edu 2006-01-09T13:28:47-05:00 re: rutgers changes for 2.1: accessservlet: aliases, content view, support for html editors, reordering in resources
http://mbox.dr-chuck.net/sakai.devel/420/421 6457
    lance@indiana.edu 2006-01-09T13:45:57-05:00 re: tools extending javax.sql.basedatasource
http://mbox.dr-chuck.net/sakai.devel/421/422 11029
    nuno@ufp.pt 2006-01-09T14:05:25+00:00 re: staleobjectstateexception updating profile
http://mbox.dr-chuck.net/sakai.devel/422/423 7446
    ggolden@umich.edu 2006-01-09T14:27:39-05:00 re: tools extending javax.sql.basedatasource
http://mbox.dr-chuck.net/sakai.devel/423/424 8927
    dhorwitz@ched.uct.ac.za 2006-01-09T14:29:47+02:00 re: staleobjectstateexception updating profile
http://mbox.dr-chuck.net/sakai.devel/424/425 4468
    nuno@ufp.pt 2006-01-09T14:44:20+00:00 re: staleobjectstateexception updating profile
http://mbox.dr-chuck.net/sakai.devel/425/426 5667
    ys2n@virginia.edu 

http://mbox.dr-chuck.net/sakai.devel/476/477 4870
    csev@umich.edu 2006-01-11T11:49:13-05:00 sakai development site on collab.
http://mbox.dr-chuck.net/sakai.devel/477/478 3442
    nuno@ufp.pt 2006-01-11T12:36:06+00:00 re: strange sakai behavior
http://mbox.dr-chuck.net/sakai.devel/478/479 4461
    csev@umich.edu 2006-01-11T12:53:06-05:00 re: rutgers changes for 2.1: accessservlet: aliases, content view, support for html editors, reordering in resources
http://mbox.dr-chuck.net/sakai.devel/479/480 3469
    csev@umich.edu 2006-01-11T13:01:06-05:00 re: rutgers changes for 2.1: accessservlet: aliases, content view, support for html editors, reordering in resources
http://mbox.dr-chuck.net/sakai.devel/480/481 3792
    tomlewis@u.washington.edu 2006-01-11T13:47:18-08:00 sakai community practices work group charter: please comment
http://mbox.dr-chuck.net/sakai.devel/481/482 3787
    tomlewis@u.washington.edu 2006-01-11T13:47:18-08:00 sakai community practices work group charter: please co

#### Segundo código "gmodel.py"

In [2]:
import sqlite3
import time
import re
import zlib
from datetime import datetime, timedelta

# Not all systems have this
try:
    import dateutil.parser as parser
except:
    pass

dnsmapping = dict()
mapping = dict()

def fixsender(sender,allsenders=None) :
    global dnsmapping
    global mapping
    if sender is None : return None
    sender = sender.strip().lower()
    sender = sender.replace('<','').replace('>','')

    # Check if we have a hacked gmane.org from address
    if allsenders is not None and sender.endswith('gmane.org') :
        pieces = sender.split('-')
        realsender = None
        for s in allsenders:
            if s.startswith(pieces[0]) :
                realsender = sender
                sender = s
                # print(realsender, sender)
                break
        if realsender is None :
            for s in mapping:
                if s.startswith(pieces[0]) :
                    realsender = sender
                    sender = mapping[s]
                    # print(realsender, sender)
                    break
        if realsender is None : sender = pieces[0]

    mpieces = sender.split("@")
    if len(mpieces) != 2 : return sender
    dns = mpieces[1]
    x = dns
    pieces = dns.split(".")
    if dns.endswith(".edu") or dns.endswith(".com") or dns.endswith(".org") or dns.endswith(".net") :
        dns = ".".join(pieces[-2:])
    else:
        dns = ".".join(pieces[-3:])
    # if dns != x : print(x,dns)
    # if dns != dnsmapping.get(dns,dns) : print(dns,dnsmapping.get(dns,dns))
    dns = dnsmapping.get(dns,dns)
    return mpieces[0] + '@' + dns

def parsemaildate(md) :
    # See if we have dateutil
    try:
        pdate = parser.parse(md)
        test_at = pdate.isoformat()
        return test_at
    except:
        pass

    # Non-dateutil version - we try our best

    pieces = md.split()
    notz = " ".join(pieces[:4]).strip()

    # Try a bunch of format variations - strptime() is *lame*
    dnotz = None
    for form in [ '%d %b %Y %H:%M:%S', '%d %b %Y %H:%M:%S',
        '%d %b %Y %H:%M', '%d %b %Y %H:%M', '%d %b %y %H:%M:%S',
        '%d %b %y %H:%M:%S', '%d %b %y %H:%M', '%d %b %y %H:%M' ] :
        try:
            dnotz = datetime.strptime(notz, form)
            break
        except:
            continue

    if dnotz is None :
        # print('Bad Date:',md)
        return None

    iso = dnotz.isoformat()

    tz = "+0000"
    try:
        tz = pieces[4]
        ival = int(tz) # Only want numeric timezone values
        if tz == '-0000' : tz = '+0000'
        tzh = tz[:3]
        tzm = tz[3:]
        tz = tzh+":"+tzm
    except:
        pass

    return iso+tz

# Parse out the info...
def parseheader(hdr, allsenders=None):
    if hdr is None or len(hdr) < 1 : return None
    sender = None
    x = re.findall('\nFrom: .* <(\S+@\S+)>\n', hdr)
    if len(x) >= 1 :
        sender = x[0]
    else:
        x = re.findall('\nFrom: (\S+@\S+)\n', hdr)
        if len(x) >= 1 :
            sender = x[0]

    # normalize the domain name of Email addresses
    sender = fixsender(sender, allsenders)

    date = None
    y = re.findall('\nDate: .*, (.*)\n', hdr)
    sent_at = None
    if len(y) >= 1 :
        tdate = y[0]
        tdate = tdate[:26]
        try:
            sent_at = parsemaildate(tdate)
        except Exception as e:
            # print('Date ignored ',tdate, e)
            return None

    subject = None
    z = re.findall('\nSubject: (.*)\n', hdr)
    if len(z) >= 1 : subject = z[0].strip().lower()

    guid = None
    z = re.findall('\nMessage-ID: (.*)\n', hdr)
    if len(z) >= 1 : guid = z[0].strip().lower()

    if sender is None or sent_at is None or subject is None or guid is None :
        return None
    return (guid, sender, subject, sent_at)

conn = sqlite3.connect('index.sqlite')
cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS Messages ''')
cur.execute('''DROP TABLE IF EXISTS Senders ''')
cur.execute('''DROP TABLE IF EXISTS Subjects ''')
cur.execute('''DROP TABLE IF EXISTS Replies ''')

cur.execute('''CREATE TABLE IF NOT EXISTS Messages
    (id INTEGER PRIMARY KEY, guid TEXT UNIQUE, sent_at INTEGER,
     sender_id INTEGER, subject_id INTEGER,
     headers BLOB, body BLOB)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Senders
    (id INTEGER PRIMARY KEY, sender TEXT UNIQUE)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Subjects
    (id INTEGER PRIMARY KEY, subject TEXT UNIQUE)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Replies
    (from_id INTEGER, to_id INTEGER)''')

conn_1 = sqlite3.connect('mapping.sqlite')
cur_1 = conn_1.cursor()

cur_1.execute('''SELECT old,new FROM DNSMapping''')
for message_row in cur_1 :
    dnsmapping[message_row[0].strip().lower()] = message_row[1].strip().lower()

mapping = dict()
cur_1.execute('''SELECT old,new FROM Mapping''')
for message_row in cur_1 :
    old = fixsender(message_row[0])
    new = fixsender(message_row[1])
    mapping[old] = fixsender(new)

# Done with mapping.sqlite
conn_1.close()

# Open the main content (Read only)
conn_1 = sqlite3.connect('file:content.sqlite?mode=ro', uri=True)
cur_1 = conn_1.cursor()

allsenders = list()
cur_1.execute('''SELECT email FROM Messages''')
for message_row in cur_1 :
    sender = fixsender(message_row[0])
    if sender is None : continue
    if 'gmane.org' in sender : continue
    if sender in allsenders: continue
    allsenders.append(sender)

print("Loaded allsenders",len(allsenders),"and mapping",len(mapping),"dns mapping",len(dnsmapping))

cur_1.execute('''SELECT headers, body, sent_at
    FROM Messages ORDER BY sent_at''')

senders = dict()
subjects = dict()
guids = dict()

count = 0

for message_row in cur_1 :
    hdr = message_row[0]
    parsed = parseheader(hdr, allsenders)
    if parsed is None: continue
    (guid, sender, subject, sent_at) = parsed

    # Apply the sender mapping
    sender = mapping.get(sender,sender)

    count = count + 1
    if count % 250 == 1 : print(count,sent_at, sender)
    # print(guid, sender, subject, sent_at)

    if 'gmane.org' in sender:
        print("Error in sender ===", sender)

    sender_id = senders.get(sender,None)
    subject_id = subjects.get(subject,None)
    guid_id = guids.get(guid,None)

    if sender_id is None :
        cur.execute('INSERT OR IGNORE INTO Senders (sender) VALUES ( ? )', ( sender, ) )
        conn.commit()
        cur.execute('SELECT id FROM Senders WHERE sender=? LIMIT 1', ( sender, ))
        try:
            row = cur.fetchone()
            sender_id = row[0]
            senders[sender] = sender_id
        except:
            print('Could not retrieve sender id',sender)
            break
    if subject_id is None :
        cur.execute('INSERT OR IGNORE INTO Subjects (subject) VALUES ( ? )', ( subject, ) )
        conn.commit()
        cur.execute('SELECT id FROM Subjects WHERE subject=? LIMIT 1', ( subject, ))
        try:
            row = cur.fetchone()
            subject_id = row[0]
            subjects[subject] = subject_id
        except:
            print('Could not retrieve subject id',subject)
            break
    # print(sender_id, subject_id)
    cur.execute('INSERT OR IGNORE INTO Messages (guid,sender_id,subject_id,sent_at,headers,body) VALUES ( ?,?,?,datetime(?),?,? )',
            ( guid, sender_id, subject_id, sent_at,
            zlib.compress(message_row[0].encode()), zlib.compress(message_row[1].encode())) )
    conn.commit()
    cur.execute('SELECT id FROM Messages WHERE guid=? LIMIT 1', ( guid, ))
    try:
        row = cur.fetchone()
        message_id = row[0]
        guids[guid] = message_id
    except:
        print('Could not retrieve guid id',guid)
        break

cur.close()
cur_1.close()


Loaded allsenders 106 and mapping 29 dns mapping 1
1 2005-12-08T23:34:30-06:00 ggolden22@mac.com
251 2005-12-22T10:03:20-08:00 tpamsler@ucdavis.edu


#### Tercer código "gbasic.py"

In [ ]:
import sqlite3
import time
import zlib

howmany = int(input("How many to dump? "))

conn = sqlite3.connect('index.sqlite')
cur = conn.cursor()

cur.execute('SELECT id, sender FROM Senders')
senders = dict()
for message_row in cur :
    senders[message_row[0]] = message_row[1]

cur.execute('SELECT id, subject FROM Subjects')
subjects = dict()
for message_row in cur :
    subjects[message_row[0]] = message_row[1]

# cur.execute('SELECT id, guid,sender_id,subject_id,headers,body FROM Messages')
cur.execute('SELECT id, guid,sender_id,subject_id,sent_at FROM Messages')
messages = dict()
for message_row in cur :
    messages[message_row[0]] = (message_row[1],message_row[2],message_row[3],message_row[4])

print("Loaded messages=",len(messages),"subjects=",len(subjects),"senders=",len(senders))

sendcounts = dict()
sendorgs = dict()
for (message_id, message) in list(messages.items()):
    sender = message[1]
    sendcounts[sender] = sendcounts.get(sender,0) + 1
    pieces = senders[sender].split("@")
    if len(pieces) != 2 : continue
    dns = pieces[1]
    sendorgs[dns] = sendorgs.get(dns,0) + 1

print('')
print('Top',howmany,'Email list participants')

x = sorted(sendcounts, key=sendcounts.get, reverse=True)
for k in x[:howmany]:
    print(senders[k], sendcounts[k])
    if sendcounts[k] < 10 : break

print('')
print('Top',howmany,'Email list organizations')

x = sorted(sendorgs, key=sendorgs.get, reverse=True)
for k in x[:howmany]:
    print(k, sendorgs[k])
    if sendorgs[k] < 10 : break


#### Códigos "gline.py" y "gword.py" son ejecutados para la visualización, dichos códigos están en la carpeta